# Attempt to Replicate the Step-for-Step Process with Stock Price Dataset from Kaggle

## Create df_close and df_open

In [5]:
import random
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
import statsmodels.api as sm
from sklearn.svm import LinearSVR, SVR
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import random
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
import statsmodels.api as sm
from sklearn.svm import LinearSVR, SVR
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.model_selection import RepeatedKFold, cross_val_score, train_test_split
from statsmodels.tsa.api import AutoReg, arma_order_select_ic, ARIMA, ARDL, ardl_select_order
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import RandomizedSearchCV, RepeatedKFold, cross_val_score, train_test_split

import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import RandomizedSearchCV, RepeatedKFold, cross_val_score, train_test_split

import warnings
warnings.filterwarnings('ignore')

In [6]:
# Unzip historical stock prices dataset
!unzip historical_stock_prices.csv.zip

Archive:  historical_stock_prices.csv.zip
replace historical_stock_prices.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
# Read data
df = pd.read_csv('historical_stock_prices.csv')
df_spx = pd.read_csv('sp500_companies.csv')

In [8]:
df_spx.head()

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
0,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,254.49,3846819807232,1.346610e+11,0.061,Cupertino,CA,United States,164000.0,"Apple Inc. designs, manufactures, and markets ...",0.069209
1,NMS,NVDA,NVIDIA Corporation,NVIDIA Corporation,Technology,Semiconductors,134.70,3298803056640,6.118400e+10,1.224,Santa Clara,CA,United States,29600.0,NVIDIA Corporation provides graphics and compu...,0.059350
2,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software - Infrastructure,436.60,3246068596736,1.365520e+11,0.160,Redmond,WA,United States,228000.0,Microsoft Corporation develops and supports so...,0.058401
3,NMS,AMZN,"Amazon.com, Inc.","Amazon.com, Inc.",Consumer Cyclical,Internet Retail,224.92,2365033807872,1.115830e+11,0.110,Seattle,WA,United States,1551000.0,"Amazon.com, Inc. engages in the retail sale of...",0.042550
4,NMS,GOOGL,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,191.41,2351625142272,1.234700e+11,0.151,Mountain View,CA,United States,181269.0,Alphabet Inc. offers various products and plat...,0.042309


In [9]:
# df only has ticker entries from Symbol in df_spx
df = df[df['ticker'].isin(df_spx['Symbol'])]

In [10]:
# Eliminate all dates before 1990
df = df[df['date'] >= '1990-01-01']

In [11]:
# See all entries in the array
tickers = df['ticker'].unique()

# View all entries in tickers without ...
tickers = [ticker for ticker in tickers]
print(tickers)

['AAPL', 'RJF', 'MHK', 'SBUX', 'PFE', 'PFG', 'CLX', 'AIG', 'OMC', 'RSG', 'RF', 'MKC', 'ARE', 'EBAY', 'CME', 'RL', 'AIZ', 'AJG', 'HBAN', 'CMG', 'PGR', 'MLM', 'CMI', 'SBAC', 'ODFL', 'IPG', 'CTSH', 'ROST', 'AMAT', 'KLAC', 'GWW', 'IDXX', 'NTRS', 'MMC', 'XYL', 'CSGP', 'CMS', 'ANET', 'RMD', 'ALGN', 'MMM', 'TECH', 'LDOS', 'SO', 'IQV', 'VICI', 'NCLH', 'EXPD', 'ALB', 'PHM', 'EXPE', 'CNC', 'IRM', 'MCHP', 'ZTS', 'ALL', 'CNP', 'HSIC', 'ADSK', 'ANSS', 'NTAP', 'GNRC', 'MOH', 'TTWO', 'MOS', 'OKE', 'ATO', 'INCY', 'AMD', 'TSLA', 'AME', 'TPL', 'MPC', 'FOXA', 'CTAS', 'TPR', 'AMZN', 'DXCM', 'AMP', 'COF', 'ITW', 'AMT', 'NFLX', 'PKG', 'COO', 'AVB', 'MRK', 'PAYC', 'GLW', 'EPAM', 'ROL', 'COP', 'VZ', 'NDSN', 'COR', 'ROK', 'IVZ', 'PLD', 'JNPR', 'ROP', 'ALLE', 'CPB', 'XOM', 'GOOG', 'WM', 'PAYX', 'PYPL', 'REGN', 'ISRG', 'AVY', 'MSI', 'FRT', 'WY', 'AON', 'SWKS', 'CPT', 'AOS', 'CPRT', 'STE', 'GPC', 'APA', 'WTW', 'MTB', 'AWK', 'MTD', 'GPN', 'ZBRA', 'DLTR', 'APD', 'STT', 'TRV', 'CDNS', 'APH', 'STX', 'STZ', 'TSN', 'AX

In [12]:
# Date count
df.shape

(2595132, 8)

In [13]:
# Export df as a new csv
df.to_csv('df_spx.csv', index=False)

In [14]:
# Make new data frame where tickers are columns, and opening prices are listed by date
df_open = df.pivot(index='date', columns='ticker', values='open')
df_close = df.pivot(index='date', columns='ticker', values='close')

In [15]:
# Export df_open and df_close as csv files
df_open.to_csv('df_open.csv')
df_close.to_csv('df_close.csv')

In [16]:
# Change nulls to 0
df_open = df_open.fillna(0)
df_close = df_close.fillna(0)

In [17]:
df_open.head()

ticker,A,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,0.0,1.258929,0.0,3.829686,0.0,0.0,1.265625,1.645833,8.538174,4.820343,...,0.0,27.375,0.0,19.8125,12.46875,0.0,0.0,0.0,0.0,0.0
1990-01-03,0.0,1.357143,0.0,3.892812,0.0,0.0,1.281250,1.583333,8.538174,4.944259,...,0.0,28.375,0.0,20.1250,12.46875,0.0,0.0,0.0,0.0,0.0
1990-01-04,0.0,1.366071,0.0,3.906841,0.0,0.0,1.343750,1.500000,8.538174,4.919476,...,0.0,27.500,0.0,20.0625,12.34375,0.0,0.0,0.0,0.0,0.0
1990-01-05,0.0,1.348214,0.0,3.878784,0.0,0.0,1.375000,1.500000,8.491771,4.857518,...,0.0,27.125,0.0,19.5625,12.25000,0.0,0.0,0.0,0.0,0.0
1990-01-08,0.0,1.339286,0.0,3.815657,0.0,0.0,1.406250,1.500000,8.027740,4.832734,...,0.0,27.125,0.0,19.3750,12.18750,0.0,0.0,0.0,0.0,0.0


## Part 1: Dataset creation with Non-Overlapping Testing Period

Split the data with a 4-year window, 1-year stride, each study period is approxiamtely 756 days, each trading period is 252 days.

Initial paper showed 26 study periods, our results show 25 due to absence of days in the dataset.

Generated with ChatGPT

In [18]:
import pandas as pd

# Load data
df_open = pd.read_csv("df_open.csv", parse_dates=["date"], index_col="date")
df_close = pd.read_csv("df_close.csv", parse_dates=["date"], index_col="date")  # Ensure df_close is available

# Ensure data is sorted by date
df_open.sort_index(inplace=True)
df_close.sort_index(inplace=True)

# Study period parameters
window_size = 4 * 252  # 4 years in trading days
stride = 1 * 252  # 1-year stride
train_size = 3 * 252  # 3-year training
trade_size = 1 * 252  # 1-year trading

study_periods = []

for start in range(0, len(df_open) - window_size + 1, stride):
    train_start = start
    train_end = start + train_size
    trade_start = train_end
    trade_end = trade_start + trade_size

    train_open = df_open.iloc[train_start:train_end]
    trade_open = df_open.iloc[trade_start:trade_end]
    train_close = df_close.iloc[train_start:train_end]
    trade_close = df_close.iloc[trade_start:trade_end]

    study_periods.append({
        "train_open": train_open,
        "trade_open": trade_open,
        "train_close": train_close,
        "trade_close": trade_close
    })

# Example: Access the first study period
first_period = study_periods[0]
print("First study period - Training Open Prices:\n", first_period["train_open"].head())



First study period - Training Open Prices:
              A      AAPL  ABBV       ABT  ACGL  ACN      ADBE       ADI  \
date                                                                      
1990-01-02 NaN  1.258929   NaN  3.829686   NaN  NaN  1.265625  1.645833   
1990-01-03 NaN  1.357143   NaN  3.892812   NaN  NaN  1.281250  1.583333   
1990-01-04 NaN  1.366071   NaN  3.906841   NaN  NaN  1.343750  1.500000   
1990-01-05 NaN  1.348214   NaN  3.878784   NaN  NaN  1.375000  1.500000   
1990-01-08 NaN  1.339286   NaN  3.815657   NaN  NaN  1.406250  1.500000   

                 ADM       ADP  ...  WTW      WY  WYNN      XEL       XOM  \
date                            ...                                         
1990-01-02  8.538174  4.820343  ...  NaN  27.375   NaN  19.8125  12.46875   
1990-01-03  8.538174  4.944259  ...  NaN  28.375   NaN  20.1250  12.46875   
1990-01-04  8.538174  4.919476  ...  NaN  27.500   NaN  20.0625  12.34375   
1990-01-05  8.491771  4.857518  ...  NaN  27.

## Part 2.1: Features Selection

In [19]:
k = 10
first_study = study_periods[0]
trade_open = first_study["trade_open"]
trade_close = first_study["trade_close"]

# Choosing the last available day in the trading period as τ
tau = trade_open.index[-1]

# Extracting opening prices for τ and adjusted closing prices for τ-1
op_tau = trade_open.loc[tau]
cp_tau_minus_1 = trade_close.loc[trade_open.index[-2]]  # Previous day's closing

# Compute intraday return irτ,0 = (cpτ / opτ) - 1
intraday_returns = (cp_tau_minus_1 / op_tau) - 1

# Select top k highest and lowest intraday return stocks
top_k_stocks = intraday_returns.nlargest(k)
bottom_k_stocks = intraday_returns.nsmallest(k)

# Output the results
print("Top k stocks with highest intraday returns:")
print(top_k_stocks)

print("\nBottom k stocks with lowest intraday returns:")
print(bottom_k_stocks)


Top k stocks with highest intraday returns:
ROP     0.067797
VRTX    0.045455
BAX     0.032967
TRMB    0.028571
AAPL    0.027523
GILD    0.022222
BIIB    0.022222
NDSN    0.019802
UDR     0.019802
SYK     0.018182
dtype: float64

Bottom k stocks with lowest intraday returns:
JBL    -0.032258
CSCO   -0.031873
INCY   -0.027778
KIM    -0.025830
TYL    -0.025641
TROW   -0.017241
EXPD   -0.016667
JKHY   -0.016667
PFE    -0.016605
CTAS   -0.015625
dtype: float64


## Part 2.2: Features Generation

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler

# Load data
df_open = pd.read_csv("df_open.csv", parse_dates=["date"], index_col="date")
df_close = pd.read_csv("df_close.csv", parse_dates=["date"], index_col="date")

# Ensure data is sorted by date
df_open.sort_index(inplace=True)
df_close.sort_index(inplace=True)

# Debug: Check if column names are correct
print(f"df_open columns: {df_open.columns}")
print(f"df_close columns: {df_close.columns}")

# Compute features for each stock
intraday_return = (df_close / df_open) - 1  # ir(s)_t,1
close_to_close_return = df_close.pct_change()  # cr(s)_t,1
open_to_open_return = df_open.pct_change()  # or(s)_t,1

# Debug: Check shape before merging
print(f"Intraday return shape: {intraday_return.shape}")
print(f"Close-to-close return shape: {close_to_close_return.shape}")
print(f"Open-to-open return shape: {open_to_open_return.shape}")

# Merge features into a single DataFrame
df_features = pd.concat(
    [intraday_return, close_to_close_return, open_to_open_return],
    axis=1,
    keys=['ir', 'cr', 'or']
)

# Debug: Check structure after merging
print(f"df_features columns:\n{df_features.columns}")

# Ensure df_features has MultiIndex format
if not isinstance(df_features.columns, pd.MultiIndex):
    print("Fixing df_features column format...")
    df_features.columns = pd.MultiIndex.from_tuples(df_features.columns)

# Remove NaNs caused by pct_change()
df_features.dropna(inplace=True)

# Debug: Show sample data
print("Sample df_features after merging and cleaning:")
print(df_features.head())

# Apply Robust Scaler standardization
def robust_scale(series):
    Q1, Q2, Q3 = series.quantile([0.25, 0.5, 0.75])
    return (series - Q2) / (Q3 - Q1) if (Q3 - Q1) != 0 else series  # Avoid division by zero

df_scaled = df_features.groupby(axis=1, level=1).apply(lambda x: x.apply(robust_scale))

# Debug: Ensure we still have data
if df_scaled.empty:
    print("Error: No data available after feature scaling.")
    exit()

# Debug: Show scaled data
print("Sample df_scaled:")
print(df_scaled.head())

# Generate overlapping sequences of 240 time steps for training
sequence_length = 240
X, y = [], []

# Get all stock tickers
# Get actual stock tickers
stock_tickers = df_scaled.columns.get_level_values(1).unique().tolist()
print(f"Stock tickers found: {stock_tickers}")

X, y = [], []
sequence_length = 240

for stock in stock_tickers:
    stock_data = df_scaled.loc[:, (slice(None), stock)].values  # Extracts (ir, cr, or) features
    print(f"Processing stock: {stock}")
    print(f"Stock data shape: {stock_data.shape}")

    if stock_data.shape[0] <= sequence_length:
        print(f"Skipping stock {stock} due to insufficient data ({stock_data.shape[0]} rows).")
        continue  # Skip if not enough data

    # Generate training sequences
    for t in range(sequence_length, len(stock_data) - 1):  # Ensure 241st return exists
        X.append(stock_data[t-sequence_length:t])  # Last 240 timesteps
        y.append(1 if intraday_return.iloc[t+1][stock] > 0 else -1)  # Predict direction

X = np.array(X)
y = np.array(y)

print(f"Input shape: {X.shape} (samples, time steps, features)")
print(f"Target shape: {y.shape} (samples,)")

if X.shape[0] == 0:
    print("Error: No training samples were generated. Ensure your dataset has enough rows.")

df_open columns: Index(['A', 'AAPL', 'ABBV', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP',
       ...
       'WTW', 'WY', 'WYNN', 'XEL', 'XOM', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZTS'],
      dtype='object', length=458)
df_close columns: Index(['A', 'AAPL', 'ABBV', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP',
       ...
       'WTW', 'WY', 'WYNN', 'XEL', 'XOM', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZTS'],
      dtype='object', length=458)
Intraday return shape: (7220, 458)
Close-to-close return shape: (7220, 458)
Open-to-open return shape: (7220, 458)
df_features columns:
MultiIndex([('ir',    'A'),
            ('ir', 'AAPL'),
            ('ir', 'ABBV'),
            ('ir',  'ABT'),
            ('ir', 'ACGL'),
            ('ir',  'ACN'),
            ('ir', 'ADBE'),
            ('ir',  'ADI'),
            ('ir',  'ADM'),
            ('ir',  'ADP'),
            ...
            ('or',  'WTW'),
            ('or',   'WY'),
            ('or', 'WYNN'),
            ('or',  'XEL'),
            ('o

We run into a roadblock.  The input shape and target shape should ideally be 241 features to train and test split, but our dataset consists of missing days and lots of null values which could interrupt time series modeling lookbacks.  

# Verify datasets are able to work with given code

## Generated with ChatGPT

In [12]:
import numpy as np
import pandas as pd
import random
import time
import os
import warnings
from sklearn.preprocessing import OneHotEncoder, RobustScaler

import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dropout, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.models import Model

# -------------------- Setup --------------------
warnings.filterwarnings("ignore")
SEED = 9
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)

# Load data
SP500_df = pd.read_csv('SPXconst.csv')
df_open = pd.read_csv('Open-1990.csv')
df_close = pd.read_csv('Close-1990.csv')

# Clean and parse SP500 constituents
constituents = {
    '-'.join(col.split('/')[::-1]): set(SP500_df[col].dropna())
    for col in SP500_df.columns
}
test_year = 1991
start_year = max(1990, test_year - 3)
months = [
    f"{t}-0{m}" if m < 10 else f"{t}-{m}"
    for t in range(start_year, test_year) for m in range(1, 13)
]
valid_months = [m for m in months if m in constituents]
constituents_train = {
    test_year: set(i for sublist in [list(constituents[m]) for m in valid_months] for i in sublist)
}

# Create necessary folders
model_folder = 'models-Intraday-240-3-LSTM'
result_folder = 'results-Intraday-240-3-LSTM'
for directory in [model_folder, result_folder]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Dummy statistics class
class DummyStatistics:
    def __init__(self, data):
        self.data = data

    def mean(self):
        return self.data.mean()

    def sharpe(self):
        return self.data.mean() / self.data.std() * np.sqrt(252)

    def shortreport(self):
        print("Mean:", self.mean())
        print("Sharpe Ratio:", self.sharpe())

# -------------------- Model and Helper Functions --------------------
def makeLSTM():
    inputs = Input(shape=(240, 3))
    x = LSTM(25, return_sequences=False)(inputs)
    x = Dropout(0.1)(x)
    outputs = Dense(2, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model

def callbacks_req(model_type='LSTM'):
    csv_logger = CSVLogger(f"{model_folder}/training-log-{model_type}-{test_year}.csv")
    filepath = f"{model_folder}/model-{model_type}-{test_year}-E{{epoch:02d}}.h5"
    model_checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=False)
    earlyStopping = EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
    return [csv_logger, earlyStopping, model_checkpoint]

def reshaper(arr):
    arr = np.array(np.split(arr, 3, axis=1))
    arr = np.swapaxes(arr, 0, 1)
    arr = np.swapaxes(arr, 1, 2)
    return arr

def create_label(df_open, df_close, perc=[0.5, 0.5]):
    if not np.all(df_close.iloc[:, 0] == df_open.iloc[:, 0]):
        raise ValueError("Date Index mismatch between Open and Close data")
    perc = [0.] + list(np.cumsum(perc))
    label = (df_close.iloc[:, 1:] / df_open.iloc[:, 1:] - 1).apply(
        lambda x: pd.qcut(x.rank(method='first'), perc, labels=False), axis=1)
    return label[1:]

def create_stock_data(df_open, df_close, st, label, m=240):
    st_data = pd.DataFrame()
    st_data['Date'] = df_close['Date']
    st_data['Name'] = [st] * len(st_data)
    daily_change = df_close[st] / df_open[st] - 1
    for k in range(m)[::-1]:
        st_data[f'IntraR{k}'] = daily_change.shift(k)

    nextday_ret = (np.array(df_open[st][1:]) / np.array(df_close[st][:-1]) - 1)
    nextday_ret = pd.Series(list(nextday_ret) + [np.nan])
    for k in range(m)[::-1]:
        st_data[f'NextR{k}'] = nextday_ret.shift(k)

    close_change = df_close[st].pct_change()
    for k in range(m)[::-1]:
        st_data[f'CloseR{k}'] = close_change.shift(k)

    st_data['IntraR-future'] = daily_change.shift(-1)
    st_data['label'] = list(label[st]) + [np.nan]
    st_data['Month'] = df_close['Date'].str[:7]
    st_data = st_data.dropna()

    trade_year = st_data['Month'].str[:4]
    st_data = st_data.drop(columns=['Month'])
    st_train_data = st_data[trade_year < str(test_year)]
    st_test_data = st_data[trade_year == str(test_year)]
    return np.array(st_train_data), np.array(st_test_data)

def scalar_normalize(train_data, test_data):
    scaler = RobustScaler()
    scaler.fit(train_data[:, 2:-2])
    train_data[:, 2:-2] = scaler.transform(train_data[:, 2:-2])
    test_data[:, 2:-2] = scaler.transform(test_data[:, 2:-2])

import matplotlib.pyplot as plt

def trainer(train_data, test_data):
    np.random.shuffle(train_data)
    train_x, train_y, train_ret = train_data[:, 2:-2], train_data[:, -1], train_data[:, -2]

    # Convert train_x to numeric if it contains object dtype
    train_x = train_x.astype(np.float32)  # or np.float64 if necessary

    train_x = reshaper(train_x)
    train_y = np.reshape(train_y, (-1, 1))
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(train_y)
    enc_y = enc.transform(train_y).toarray()

    model = makeLSTM()
    callbacks = callbacks_req()

    # Train the model and capture history
    history = model.fit(
        train_x,
        enc_y,
        epochs=1000,
        validation_split=0.2,
        callbacks=callbacks,
        batch_size=512,
        verbose=0
    )

    # Predict on test data
    dates = list(set(test_data[:, 0]))
    predictions = {}
    for day in dates:
        test_d = test_data[test_data[:, 0] == day]
        # Ensure test_d is numeric before reshaping and prediction
        test_d = test_d[:, 2:-2].astype(np.float32)  # or np.float64 if necessary
        test_d = reshaper(test_d)
        predictions[day] = model.predict(test_d)[:, 1]
    return model, predictions
def simulate(test_data, predictions):
    rets = pd.DataFrame([], columns=['Long', 'Short'])
    k = 10
    for day in sorted(predictions.keys()):
        preds = predictions[day]
        test_returns = test_data[test_data[:, 0] == day][:, -2]
        top_preds = preds.argsort()[-k:][::-1]
        worst_preds = preds.argsort()[:k][::-1]
        rets.loc[day] = [
            np.mean(test_returns[top_preds]),
            -np.mean(test_returns[worst_preds])
        ]
    print('Result:', rets.mean())
    return rets

# -------------------- Run the Process --------------------
label = create_label(df_open, df_close)
stock_names = sorted(list(constituents[str(test_year - 1) + '-12']))
train_data, test_data = [], []

for st in stock_names:
    try:
        st_train_data, st_test_data = create_stock_data(df_open, df_close, st, label)
        train_data.append(st_train_data)
        test_data.append(st_test_data)
    except Exception:
        continue  # Skip any stock with missing data

train_data = np.concatenate(train_data)
test_data = np.concatenate(test_data)

scalar_normalize(train_data, test_data)
model, predictions = trainer(train_data, test_data)
returns = simulate(test_data, predictions)
returns.to_csv(f"{result_folder}/avg_daily_rets-{test_year}.csv")

# Save and print result summary
result = DummyStatistics(returns.sum(axis=1))
print('\nAverage returns prior to transaction charges')
result.shortreport()

with open(f"{result_folder}/avg_returns.txt", "a") as myfile:
    res = '-' * 30 + '\n'
    res += f"{test_year}\n"
    res += f"Mean = {result.mean()}\n"
    res += f"Sharpe = {result.sharpe()}\n"
    res += '-' * 30 + '\n'
    myfile.write(res)


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
16/16 ━━━━━━━━━━━━━━━━━━